In [1]:
%reload_ext autoreload
# for auto reloading modules without kernel restart

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import timeit
from dataclasses import dataclass       # C like structure
import glob                             # finds all the pathnames matching specified pattern
!python --version
print('pandas version: ' + pd.__version__)
print('numpy version: ' + np.__version__)

Python 3.9.7
pandas version: 1.3.4
numpy version: 1.20.3


In [3]:
# Plot related packages,%matplotlib notebook makes plots in jupyter interactive
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.dates as md
import datetime as dt
import time

In [4]:
import sys
sys.path.append('C:\\Users\\s.kaiser\\Experiments\\MuleCarData\\my_modules') # add custom module folder
import custom_plot
import data_clean

imported custom_plot.py
data_clean imported


In [5]:
df_jan = pd.read_pickle("jan15_19_2022.pkl")
df_feb = pd.read_pickle("feb22_24_2022.pkl")

In [44]:
# df_feb
# Multi step multi variate sequence foreccasting: using past observations to forecast a sequence of future observations
# https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    # axis 0: #rows, axis 1: #cols, axis 2: #3rd dim
    n_vars = 1 if type(data) is list else data.shape[1]         # shape [1] = #cols
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))                    # shifts all rows down by i
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
        print(cols)
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]


    print("col len: " + str(len(cols)))
    print(cols)
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [45]:
# df_feb['7c0.brickvoltage002_bms01']

raw = pd.DataFrame()
raw['ob1'] = [x for x in range(10)]
raw['ob2'] = [x for x in range(50, 60)]
values = raw.values

data = series_to_supervised(values, 1, 2)
#print(data)
#print(values)


[     0     1
0  NaN   NaN
1  0.0  50.0
2  1.0  51.0
3  2.0  52.0
4  3.0  53.0
5  4.0  54.0
6  5.0  55.0
7  6.0  56.0
8  7.0  57.0
9  8.0  58.0]
col len: 3
[     0     1
0  NaN   NaN
1  0.0  50.0
2  1.0  51.0
3  2.0  52.0
4  3.0  53.0
5  4.0  54.0
6  5.0  55.0
7  6.0  56.0
8  7.0  57.0
9  8.0  58.0,    0   1
0  0  50
1  1  51
2  2  52
3  3  53
4  4  54
5  5  55
6  6  56
7  7  57
8  8  58
9  9  59,      0     1
0  1.0  51.0
1  2.0  52.0
2  3.0  53.0
3  4.0  54.0
4  5.0  55.0
5  6.0  56.0
6  7.0  57.0
7  8.0  58.0
8  9.0  59.0
9  NaN   NaN]
